# Running interactive AMBER simulations using BioSimSpace

In this notebook you'll learn how to use BioSimSpace to run various molecular simulation protocols with AMBER. 

Before we get started, let's import all of the modules that will be required.

In [1]:
# Import BioSimSpace and rename it for convenience.
import BioSimSpace as BSS

# Import the handy glob function.
from glob import glob

# Import pyplot so we can plot some data.
import matplotlib.pyplot as plt

## Creating a molecular system

First of all we need to load a molecular system. Some example input files are included in the `amber` directory. Let's load one of these:

In [2]:
# Glob the input files.
files = glob("amber/ala/*")

# Create a Sire molecular system.
system = BSS.readMolecules(files)

We have now created a molecular system. The system consists of an alanine dipeptide molecule in a box of water. To show the number of molecules in the system, run:

In [3]:
system.nMolecules()

631

## Defining a simulation protocol

BioSimSpace provides functionality for defining various simulation protocols. In this notebook we will construct a typical simulation workflow that uses a sequence of simple protocols, with the output of one forming the input of the next:

1. _Minimisation:_ Energy minimisation the molecular system.
2. _Equilibration:_ Equlibration of the system to a target temperature.
3. _Production:_ Regular molecular dynamics, run at fixed temperature.

When defining a protocol we are configuring the type of simulation that we wish to run, as well as any options for the particular simulation. For example, to create a default minimisation protocol:

```python
protocol = BSS.Protcol.Minimisation()
```

This defines a minimisation protocol that uses 10000 steps. For convenience, let's use 1000 steps. This can be achieved by passing the `steps` keyword argument to the constructor, i.e.:

In [4]:
# Initialise a short minimisation protocol.
protocol = BSS.Protocol.Minimisation(steps=1000)

## Initialising a process

We now have everything that is needed to create a process object. To do so, run:

In [6]:
process = BSS.Process.Amber(system, protocol, name="minimise")

### Setting the executable

On creation, BioSimSpace searches your `PATH` for an appropriate executable for running the process. The executable that is chosen may be dependent on the available hardware and type of workfow.

To see the executable that was chosen, run:

In [7]:
process.exe()

'/home/lester/Downloads/amber16/bin/sander'

To override the executable you can pass the `exe` keyword argument to the process constructor, e.g:

```python
process = BSS.Process.Amber(system, protocol, name="minimise", exe="/path/to/custom/exe")
```

### Setting the working directory

By default, BioSimSpace runs each process inside a unique temporary workspace. This is where all of the input and configuration files will be created, as well as any of the output when the process is run.

To view the working directory and the list of autogenerated input/configuration files, run:

In [10]:
process.workDir()

'/tmp/tmpoi1v8bs1'

In [11]:
process.inputFiles()

['/tmp/tmpoi1v8bs1/minimise.amber',
 '/tmp/tmpoi1v8bs1/minimise.rst7',
 '/tmp/tmpoi1v8bs1/minimise.prm7']

When the process object is destroyed, the temporary working directory is deleted and all of the files will be lost. If you wish to specify a custom working directory, simply pass one using the `work_dir` keyword argument, e.g.:

```python
process = BSS.Process.Amber(system, protocol, name="minimise", work_dir="/path/to/custom/work/dir")
```

The directory will be created if it doesn't already exist (assuming write privileges on the path.)

### Configuring the process

For each protocol, BioSimSpace will initialise default configuration parameters appropriate to the process based on best practice in the field.

To see the list of configuration parameter strings, run:

In [14]:
process.getConfig()

['Minimisation',
 ' &cntrl',
 '  imin=1,',
 '  ntx=1,',
 '  ntxo=1,',
 '  ntpr=100,',
 '  irest=0,',
 '  maxcyc=1000,',
 '  cut=8.0,',
 ' /']

In some cases, it may be desirable to run a custom protocol. BioSimSpace provides several ways of acheiving this:

* By passing a configuration file as the `protocol` keyword argument when creating a process:

```python
process = BSS.Process.Amber(system, protocol="my_config.txt", name="minimise")
```

* By setting the configuration of an existing process:

```python
# Set the configuration from file.
process.setConfig("my_config.txt")

# Set the configuration using a list of configuration strings.
my_config = ["some config parameter string", "another config parameter string"]
process.setConfig(my_config)
```

* By adding to the configuration of an existing process:

```python
# Add using a configuration from file.
process.addToConfig("my_config.txt")

# Add a list of parameter strings to the configuration.
my_config = ["some config parameter string", "another config parameter string"]
process.addToConfig(my_config)
```

### Configuring command-line arguments

Where necessary, BioSimSpace will configure the command-line arguments needed to run the process.

To view the command-line argument string, run:

In [15]:
process.getArgString()

'-O -i minimise.amber -p minimise.prm7 -c minimise.rst7 -o stdout -r minimise.restart.crd -inf minimise.nrg'

The arguments are stored internally as an `OrderedDict` object. To view it, run:

In [16]:
process.getArgs()

OrderedDict([('-O', True),
             ('-i', 'minimise.amber'),
             ('-p', 'minimise.prm7'),
             ('-c', 'minimise.rst7'),
             ('-o', 'stdout'),
             ('-r', 'minimise.restart.crd'),
             ('-inf', 'minimise.nrg')])

BioSimSpace provides functionality for setting and manipulating the arguments. For example, to disable the overwriting of output files:

In [19]:
process.setArg('-O', False)
process.getArgs()

'-i minimise.amber -p minimise.prm7 -c minimise.rst7 -o stdout -r minimise.restart.crd -inf minimise.nrg'

Let's see how the argument string changed:

In [20]:
process.getArgString()

'-i minimise.amber -p minimise.prm7 -c minimise.rst7 -o stdout -r minimise.restart.crd -inf minimise.nrg'

The `setArg` method can be used to add a new argument, or to overwrite the value of an existing argument. There are several other methods that allow the arguments to be modified:

* setArgs(args): Overwrite all arguments with a new dictionary.
* addArgs(args): Append additional arguments.
* insertArgs(arg, value, index): Insert an argument at a specific index.
* deleteArg(arg): Delete an argument from the dictionary.
* clearArgs(): Clear all of the arguments.

If you ever get in trouble, it's easy to reset the arguments to their default values:

In [22]:
process.resetArgs()
process.getArgs()

OrderedDict([('-O', True),
             ('-i', 'minimise.amber'),
             ('-p', 'minimise.prm7'),
             ('-c', 'minimise.rst7'),
             ('-o', 'stdout'),
             ('-r', 'minimise.restart.crd'),
             ('-inf', 'minimise.nrg')])

## Running the minimisation process

Having configured the process to your liking, it's time to run a simulation.

To start the process, run:

In [32]:
process.start()

BioSimSpace has now launched a AMBER minimisation process in the background.

To see if the process is still running:

In [45]:
process.isRunning()

False

When the process has finished running we can see how many minutes it took to run.

In [46]:
process.runTime()

0.24231536428366476

We can also query the final energy of the molecular system (in kcal/mol):

In [47]:
process.getTotalEnergy()

-7422.4

Fantastic, you've just run your first simulation using BioSimSpace!

## Continuing the workflow

### Minimisation

Suppose we want to use our minimised molecular system as the input for a new equilibration protocol. First let's grab the most recent molecular system from the process. This will be the final configuration if the process has finished, or the most recent checkpoint.

In [49]:
minimised = process.getSystem()

Perhaps we would like to save this configuration to file for future use. The following command will write the configuration to disc, using the same file format as the files that were used to generate the original system. The output shows the name of the files that were written.

In [51]:
 BSS.saveMolecules("minimised", minimised, system.fileFormat())

['/home/lester/Code/BioSimSpace/demo/minimised.prm7',
 '/home/lester/Code/BioSimSpace/demo/minimised.rst7']

Next, we'll create a short equilibration protocol. Let's heat the system from 0 to 300K over a period of 0.05 nanoseconds, restraining the positions of backbone atoms in the alanine dipeptide.

In [53]:
protocol = BSS.Protocol.Equilibration(runtime=0.05, temperature_start=0, temperature_end=300, restrain_backbone=True)

We can now create a new process object using the minimised system and the equilibration protocol.

In [54]:
process = BSS.Process.Amber(minimised, protocol, name="equilibrate")

Let's check the configuration parameters for the process.

In [55]:
process.getConfig()

['Equilibration.',
 ' &cntrl',
 '  ig=-1,',
 '  ntx=1,',
 '  ntxo=1,',
 '  ntpr=100,',
 '  ntwr=500,',
 '  irest=0,',
 '  dt=0.002,',
 '  nstlim=25001,',
 '  ntc=2,',
 '  ntf=2,',
 '  ntt=3,',
 '  gamma_ln=2,',
 '  cut=8.0,',
 '  ntp=1,',
 '  pres0=1.01325,',
 '  ntr=1,',
 '  restraint_wt = 2,',
 "  restraintmask = '@CA,C,O,N',",
 '  tempi=0.00,',
 '  temp0=300.00,',
 '  nmropt=1,',
 ' /',
 "&wt TYPE='TEMP0', istep1=0, istep2=25001, value1=0.00, value2=300.00 /",
 "&wt TYPE='END' /"]

If everything looks okay, let's start the process:

In [56]:
process.start()

We can monitor the temperature and energy as the process runs.

In [58]:
print("%.2f K, %.2f kc/mol" % (process.getTemperature(), process.getTotalEnergy()))

TypeError: a float is required